# **Notebook de transfert des donnees du site nba.com depuis le site ou un fichier intermediaire**

In [3]:
import pandas as pd
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct

from TelechargementDonnees import JoueursSiteNba

# 1. Données de joueurs 
Il s'agit de transfere dans le cas des donnees de joueuer telechargees en amont, ou d'ajouter des joueurs non presents au moment d'insertion de match

>## 1.1 Enumerations
A creer a la main

In [15]:
with ct.ConnexionBdd('basket','boulot') as c : 
    pd.DataFrame({'id_position_terrain':dfJoueur.position.unique(), 
              'nom_position_terrain':['Forward ; 3 ; ailier', 'Guard ; 1 ou 2  ; meneur ou arriere', 
                                      'Center ; 5 ; Pivot' ,'Power Forward ; 4 ; ailier Fort', 
                                      'Shooting Guard ; 2 ; arriere', 'Shooting Guard ; 2 ; arriere',
                                      'Power Forward ; 4 ; ailier Fort']}).to_sql(
        'enum_position_terrain', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

> ## 1.2 Donnees chargees depuis la page internet
Là ce n'est que le premier import, on ne gère pas la mise à jour

In [5]:
#creation de la classe
pagejoueuer=JoueursSiteNba()

In [9]:
#transfert dans la Bdd
with ct.ConnexionBdd('basket','boulot') as c : 
    pagejoueuer.dfJoueurs[['nom','id_position_terrain','taille', 'poids','date_entree_nba']].to_sql(
        'joueur', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)